In [92]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv

from ML.DTGenerated import DTGenerated
import pickle

In [93]:
from generateTestingData.useGeneratedData import makeDataUsable, splitTrainTest, NewPredicter

In [94]:
MODEL_PREFIX = "gen"
PI = 4 # past importance
RBC = 5 # rows before col

In [95]:
f = makeDataUsable(read_csv("../data/generatedData.csv"), PI, RBC)
print(df.shape)
df.head()

(992, 30)


,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,...,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
1886,1319.0,50.000,50.000,50.000,50.000,3.2606,7.2518,9.8541,10.8720,2.523,...,0.006,-0.000,0.000,0.000,0.004,0.003,0.004,0.000,0.009,0.0
2893,265.0,0.436,0.025,50.000,50.000,7.3489,0.5148,2.0377,3.6029,0.128,...,-0.001,-0.059,0.246,-0.000,-0.000,0.015,-0.011,0.007,0.000,0.0
1108,541.0,50.000,48.634,38.684,3.735,26.2954,25.8733,25.3382,20.0962,4.564,...,-0.000,-0.558,-0.000,0.000,0.000,-0.019,-0.002,0.000,0.000,0.0
3614,625.5,50.000,50.000,1.318,50.000,41.0377,9.2788,6.6263,25.8055,1.192,...,0.096,0.045,0.084,0.000,-0.961,0.008,-0.022,0.001,-0.038,0.0
3813,725.0,50.000,50.000,50.000,50.000,100.0000,100.0000,100.0000,100.0000,0.016,...,-0.001,0.020,-0.001,0.028,0.001,0.023,0.003,-0.010,0.003,0.0


In [96]:
df["COL"].value_counts()

0.0    500
1.0    492
Name: COL, dtype: int64

In [97]:

# sampleSize = int(len(data[data['COL'] == 1]) * undersampleRatio)
# print("Undersampling")
# data = concat([data[data["COL"] == 0].sample(sampleSize, random_state=1), data[data["COL"] == True]])

TODO: need to either get more crash data or keep undersample below

In [98]:
df = pd.concat([df[df["COL"] == False].sample(500, random_state=1), df[df["COL"] == True]])

In [99]:
data = df.copy().drop(columns=df.columns[0], axis=1)
trainX, trainY, testX, testY = splitTrainTest(data)

Splitting at 793, total rows: 992


In [100]:
trainX.head(3)

,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,JERK2,...,asX3,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4
1302,50.0,50.0,50.0,50.0,6.1382,6.9333,8.1275,9.6133,0.099,0.035,...,0.000,0.003,0.000,-0.207,0.088,0.013,-0.0,0.036,-0.000,-0.003
2365,50.0,50.0,50.0,50.0,1.7815,3.1742,4.5489,5.8832,19.134,1.000,...,0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.0,-0.000,0.000,0.000
125,50.0,50.0,50.0,50.0,0.6156,2.6337,3.2700,8.6714,1.928,1.759,...,-0.004,0.005,0.254,-0.000,0.130,0.067,-0.0,-0.000,-0.001,0.000


## Model

In [101]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

### MLP Classifier

In [102]:
mlp_clf = NewPredicter()
mlp_trainX = mlp_clf.preProcess(trainX.copy())
mlp_testX = mlp_clf.preProcess(testX.copy())

mlp_trainY, mlp_testY = trainY.to_numpy(), testY.to_numpy()

mlp_clf.fit(mlp_trainX, mlp_trainY)

Scaler is fitted


c:\Users\Yohannes\anaconda3\envs\MasterThesis\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [103]:
mlp_pred = mlp_clf.predict(mlp_testX)
mlp_score = mlp_clf.getScore(mlp_pred, mlp_testY)

Total: 199, Collisions: 100
Accuracy: 0.6482412060301508, Precision: 0.6470588235294118, Recall: 0.66, F1: 0.6534653465346535
Confusion matrix:
[[63 36]
 [34 66]]


In [104]:
model_score = f"{mlp_score[0][0]}-{mlp_score[0][1]}-{mlp_score[1][0]}-{mlp_score[1][1]}"
mlp_clf.saveModel(f"MLPClassifier_{MODEL_PREFIX}_PI{PI}_RBC{RBC}", accuracy=model_score)

Model saved!


### Random Forest Classifier

In [105]:
rf_clf = NewPredicter(RandomForestClassifier(n_estimators=100, max_depth=10, random_state=1))
rf_trainX = rf_clf.preProcess(trainX.copy())
rf_testX = rf_clf.preProcess(testX.copy())

rf_trainY, rf_testY = trainY.to_numpy(), testY.to_numpy()

rf_clf.fit(rf_trainX, rf_trainY)

Scaler is fitted


In [106]:
rf_pred = rf_clf.predict(rf_testX)
rf_score = rf_clf.getScore(rf_pred, rf_testY)

Total: 199, Collisions: 86
Accuracy: 0.7085427135678392, Precision: 0.6372549019607843, Recall: 0.7558139534883721, F1: 0.6914893617021277
Confusion matrix:
[[76 37]
 [21 65]]


In [107]:
model_score = f"{rf_score[0][0]}-{rf_score[0][1]}-{rf_score[1][0]}-{rf_score[1][1]}"
rf_clf.saveModel(f"RandomForestClassifier_{MODEL_PREFIX}_PI{PI}_RBC{RBC}", accuracy=model_score)

Model saved!


### SVM Classifier

In [108]:
svm_clf = NewPredicter(SVC(random_state=1))
svm_trainX = svm_clf.preProcess(trainX.copy())
svm_testX = svm_clf.preProcess(testX.copy())

svm_trainY, svm_testY = trainY.to_numpy(), testY.to_numpy()

svm_clf.fit(svm_trainX, svm_trainY)

Scaler is fitted


In [109]:
svm_pred = svm_clf.predict(svm_testX)
svm_score = svm_clf.getScore(svm_pred, svm_testY)

Total: 199, Collisions: 89
Accuracy: 0.7135678391959799, Precision: 0.6568627450980392, Recall: 0.7528089887640449, F1: 0.7015706806282722
Confusion matrix:
[[75 35]
 [22 67]]


In [110]:
model_score = f"{svm_score[0][0]}-{svm_score[0][1]}-{svm_score[1][0]}-{svm_score[1][1]}"
svm_clf.saveModel(f"SVC_{MODEL_PREFIX}_PI{PI}_RBC{RBC}", accuracy=model_score)

Model saved!


### XGBoost Classifier

In [111]:
xgb_clf = NewPredicter(XGBClassifier(n_estimators=100, random_state=1))
xgb_trainX = xgb_clf.preProcess(trainX.copy())
xgb_testX = xgb_clf.preProcess(testX.copy())

xgb_trainY, xgb_testY = trainY.to_numpy(), testY.to_numpy()

Scaler is fitted


In [112]:
xgb_clf.fit(xgb_trainX, xgb_trainY)

In [113]:
len(xgb_testX[0])
xgb_pred = xgb_clf.predict(xgb_testX)
xgb_score = xgb_clf.getScore(xgb_testY, xgb_pred)

Total: 199, Collisions: 102
Accuracy: 0.7236180904522613, Precision: 0.7526881720430108, Recall: 0.6862745098039216, F1: 0.717948717948718
Confusion matrix:
[[74 23]
 [32 70]]


TODO - need more data, what is radar?

In [114]:
# model_score = f"{score[0][0]}-{score[0][1]}-{score[1][0]}-{score[1][1]}"
model_score = f"{xgb_score[0][0]}-{xgb_score[0][1]}-{xgb_score[1][0]}-{xgb_score[1][1]}"
xgb_clf.saveModel(f"XGBClassifier_{MODEL_PREFIX}_PI{PI}_RBC{RBC}", accuracy=model_score)

Model saved!
